In [ ]:
!pip install selenium==3.11.0

In [ ]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import requests
from bs4 import BeautifulSoup

import time

from konlpy.tag import Twitter
from collections import Counter

from PIL import Image 
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

### PART I. 데이터 전처리 (movie_df.csv)
: get_code()


In [ ]:
# 영화관입장권통합전산망(KOBIS)에서 역대 한국영화 박스오피스 순위 자료 수집
# (@http://www.kobis.or.kr/kobis/business/main/main.do)
# dataframe으로 저장

movie_list = pd.DataFrame([])
movied_released_years = [str(year).zfill(2) for year in range(4,21)]

for movie_released in movied_released_years:
    korean_movie = pd.read_html('KOBIS_20{}.xls'.format(movie_released),encoding='utf-8')
    korean_movie = korean_movie[1].head(10)
    movie_list = movie_list.append(korean_movie)


movie_list.to_csv('korean_movie_ranking_04_20.csv')

In [ ]:
korean_movie_list_df = pd.read_csv('korean_movie_ranking_04_20.csv')
korean_movie_list_df

In [ ]:
# '개봉일'의 yyyy-mm-dd 형식을 yyyy 형식으로 변환하여 '개봉년'에 저장
# dataframe에서 불필요한 column 제거 (순위','매출액','매출액 점유율','관객수', '스크린수', '상영횟수','대표국적','국적','배급사')
# '영화코드' column 생성

korean_movie_list_df = pd.read_csv('korean_movie_ranking_04_20.csv', index_col=0)
korean_movie_list_df['개봉년'] = korean_movie_list_df['개봉일'].str.split('-', 3, expand=True)[0]
korean_movie_list_df = korean_movie_list_df.drop(['순위','개봉일','매출액','매출액 점유율'], axis=1)
movie_df_with_03 = korean_movie_list_df.drop(['관객수', '스크린수', '상영횟수','대표국적','국적','배급사'], axis=1)
movie_df = movie_df_with_03.replace(2003, 2004)
movie_df['영화코드'] = ''


In [ ]:
# get_code() 함수 : 영화명 리스트를 input으로 영화코드 리스트를 return
# 네이버 영화 사이트의 html형식이 영화별로 상이하여 trouble shooting 기법 사용
# get_code() --> get_code_error() --> get_code_error_error()

In [ ]:
def get_code1(movie_name_list):
    
    movie_code_list = []
    driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')
    naver_movie_url = 'https://movie.naver.com/' # 네이버 영화 링크
    driver.get(naver_movie_url)
    
    for movie_name in movie_name_list:
        
        year = str(list(movie_df['개봉년'][movie_df['영화명'] == movie_name])[0])
        driver.find_element_by_id('ipt_tx_srch').send_keys(movie_name + year) # (영화이름 + 년도)로 검색
        driver.find_element_by_class_name('btn_srch').click()
        
        web = driver.page_source
        source = BeautifulSoup(web, 'html.parser')
        time.sleep(2)
        
        if source.find('p', {'class' : 'result_thumb'}) == None: # 검색 결과 페이지 최상단에 영화가 없을 때
            movie_code_list.append('000000') # 코드 대신 에러 메시지('000000') 입력
        else: # 검색 결과 페이지 최상단에 영화가 있을 때
            movie_code = source.find('p', {'class' : 'result_thumb'}).find('a').attrs['href'][-6:] 
            #consider code length : 5 or 6
            if movie_code.startswith('='):
                movie_code_list.append(movie_code[1:])
            else:
                movie_code_list.append(movie_code)
            
    driver.close()
    driver.quit()
    
    return movie_code_list

In [ ]:
def get_code2(movie_name_list): 
    
    movie_code_list = []
    driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')
    naver_movie_url = 'https://movie.naver.com/' # 네이버 영화 링크
    driver.get(naver_movie_url)
    
    for movie_name in movie_name_list:
        
        year = str(list(movie_df['개봉년'][movie_df['영화명'] == movie_name])[0])
        driver.find_element_by_id('ipt_tx_srch').send_keys(movie_name)      
        driver.find_element_by_class_name('btn_srch').click()
        
        web = driver.page_source
        source = BeautifulSoup(web, 'html.parser')
        time.sleep(2)

        # 검색 결과 최상단의 영화의 발매년도가 원하는 영화의 발매년도과 일치한다면 코드 추출
        # 네이버 영화에 발매년도가 1년 작게 등록된 경우들이 있어서 year과 year-1 모두 고려

        if (source.find('ul',{'class':'search_list_1'}).find('dd',{'class':"etc"}).find_all('a')[-1].get_text() != year) & (source.find('ul',{'class':'search_list_1'}).find('dd',{'class':"etc"}).find_all('a')[-1].get_text() != str(int(year)-1)) :
            movie_code_list.append('000001')  # 발매년도가 일치하지 않는다면 코드 대신 에러 메시지('000000') 입력
        else:
            movie_code = source.find('p', {'class' : 'result_thumb'}).find('a').attrs['href'][-6:]      
            #consider code length : 5 or 6
            if movie_code.startswith('='):
                movie_code_list.append(movie_code[1:])
            else:
                movie_code_list.append(movie_code)
            
    driver.close()
    driver.quit()
    
    return movie_code_list

In [ ]:
def get_code3(movie_name_list):#영화 이름으로 검색 후 url
    movie_code_list = []
    driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')
    naver_movie_url = 'https://movie.naver.com/' # 네이버 영화 링크
    driver.get(naver_movie_url)
    
    for movie_name in movie_name_list:
        driver.find_element_by_id('ipt_tx_srch').send_keys(movie_name)
        driver.find_element_by_class_name('btn_srch').click()
        web = driver.page_source
        source = BeautifulSoup(web, 'html.parser')
        time.sleep(3)
        year = list(korean_movie_list_df['개봉일'][korean_movie_list_df['영화명'] == movie_name])[0][:4]
        
        # 검색 결과 중 개봉년도가 일치하는 영화의 코드 추출
        # 네이버 영화에 발매년도가 1년 작게 등록된 경우들이 있어서 year과 year-1 모두 고려
        # 세 방법이 모두 통하지 않는 영화는 과감히 버림.(Try-Except)

        try:
            movies_html_list = source.find('ul',{'class':'search_list_1'}).find_all('dl')
            for i in range(len(movies_html_list)):
                movie_html = movies_html_list[i]
                if (movie_html.find('dd',{'class':"etc"}).find_all('a')[-1].get_text() == year) or (movie_html.find('dd',{'class':"etc"}).find_all('a')[-1].get_text() == str(int(year)-1)):
                    movie_code = movie_html.find('a').attrs['href'][-6:]
                    if movie_code.startswith('='):
                        movie_code_list.append(movie_code[1:])
                    else:
                        movie_code_list.append(movie_code)
                    break
        except:
            movie_code_list.append('')
            print('error in '+movie_name)
            
    driver.close()
    driver.quit()
    
    return movie_code_list

In [ ]:
# get_code1() 실행하여 movie_df에 입력
movie_df['영화코드'] = get_code1(list(movie_df['영화명']))
temp1 = movie_df

In [ ]:
# get_code1() 실행 후 error message('000000') 이 입력된 영화 목록에 대하여 get_code2() 실행 후 movie_df에 입력
error_list = get_code2(list(movie_df['영화명'][movie_df['영화코드'] =='000000']))#121
movie_df['영화코드'][movie_df['영화코드']=='000000'] = error_list
temp2 = movie_df

In [ ]:
# get_code2() 실행 후 error message('000001') 이 입력된 영화 목록에 대하여 get_code3() 실행 후 movie_df에 입력
error_list = get_code3(list(movie_df['영화명'][movie_df['영화코드'] =='000001']))#121
movie_df['영화코드'][movie_df['영화코드']=='000001'] = error_list
temp3 = movie_df

In [ ]:
# 영화명, 발매년, 영화코드 를 column으로 하는 완성된 dataframe을 'movie_df.csv'로 저장
movie_df.to_csv('movie_df.csv')

### PART II. Web Scraping (new_movie_df.csv)
: review_url(), get_review(), get_actors()

In [ ]:
new_movie_df = pd.read_csv('movie_df.csv')
new_movie_df

In [ ]:
# '영화리뷰url','영화리뷰','배우목록' column 생성

new_movie_df['영화리뷰url'] = ''
new_movie_df['영화리뷰'] = ''
new_movie_df['배우목록'] = ''
new_movie_df

In [ ]:
# 해당 영화 리뷰 최대 100개의 url 불러와서 '영화리뷰url' column에 list 형식으로 추가

def review_url(movie_name):
    idx = new_movie_df.index[new_movie_df['영화명'] == movie_name]
    movie_code = list(new_movie_df['영화코드'])[idx[0]]
    review_url_list = []

    for page in range(5): # 영화 별로 리뷰페이지 5페이지까지 web scraping
        movie_review_page_url = 'https://movie.naver.com/movie/board/review/list.nhn?st=code&sword={}&od=goodcnt&page={}'.format(movie_code,str(page+1))
        web = requests.get(movie_review_page_url)
        source = BeautifulSoup(web.text, 'html.parser')  
            
        for i in range(len(source.find_all('td',{'class':'title'}))): # 리뷰 페이지에 보여지는 리뷰 게시글들의 url 불러오기
            url = 'https://movie.naver.com/movie/board/review/' + source.find_all('td',{'class':'title'})[i].find('a').get('href')
            review_url_list.append(url)
    
    new_movie_df['영화리뷰url'][idx[0]] = review_url_list # new_movie_df에 url 추가

In [ ]:
for movie_name in new_movie_df['영화명']:
    review_url(movie_name)

In [ ]:
# savepoint

temp1 = new_movie_df
temp1

In [ ]:
# 해당 영화 리뷰들 web scraping 후 '영화리뷰' column에 str 형식으로 추가

def get_review(movie_name): # 영화    
    idx = new_movie_df.index[new_movie_df['영화명'] == movie_name]
    url_list = list(new_movie_df['영화리뷰url'])[idx[0]]
    
    review =''
    for url in url_list: # 각 리뷰(url)에서 리뷰본문.get_text()
        web = requests.get(url).content
        source = BeautifulSoup(web, 'html.parser')
        article = source.find('div', {'class': 'text_area'}).get_text() 
        review += article
    
    #replace, encode.decode ; 본문 내용에서 쓸모없는 내용 제거
    review = review.replace("\n", "")
    review = review.replace("\xa0","")
    review = review.replace("\r","")
    review = review.replace("\t","")
    review = review.replace(" .text_area #post-view {width:500px;} /*블로그에서 템플릿쓰기를 했을경우 이미지를 연속으로 붙여 넣었을때 우리쪽에서 줄바꿈이 안되는 문제를 해결하기위함*/","")
    review = review.encode('utf-8', 'ignore').decode('utf-8') # removing emoji
    #blog 주소 삭제해야함
    
    new_movie_df['영화리뷰'][idx[0]] = review

In [ ]:
for movie_name in new_movie_df['영화명']:
    get_review(movie_name)

In [ ]:
# savepoint

temp2 = new_movie_df
temp2

In [ ]:
# 해당 영화 감독 및 배우 이름 등 web scraping 후 '배우목록' column에 list 형식으로 추가

def get_actors(movie_name):
    idx = new_movie_df.index[new_movie_df['영화명'] == movie_name]
    movie_code = list(new_movie_df['영화코드'])[idx[0]]
    url = 'https://movie.naver.com/movie/board/review/list.nhn?st=code&sword={}&od=goodcnt'.format(str(movie_code))
    web = requests.get(url)
    source = BeautifulSoup(web.text, 'html.parser')
    
    actor_list = []
    
    for i in range(len(source.find('table',{'class':'info_area'}).find_all('a'))):
        actor_list.append(source.find('table',{'class':'info_area'}).find_all('a')[i].get_text())
    
    new_movie_df['배우목록'][idx[0]] = actor_list

In [ ]:
for movie_name in new_movie_df['영화명']:
    get_actors(movie_name)

In [ ]:
#savepoint

temp3 = new_movie_df
temp3

In [ ]:
# '영화명','개봉년','영화코드','영화리뷰url','영화리뷰','배우목록' 을 column으로 하는 dataframe 'new_movie_df.csv'로 저장

new_movie_df.to_csv('new_movie_df.csv')

### PART III. Wordcloud
: get_wordcounts(), get_word_dic(), wordcloud()

In [ ]:
df = pd.read_csv('new_movie_df.csv')
df

In [ ]:
# 'word_dic' column 생성

df['word_dic'] = ''

In [ ]:
# 보편적인 한국어 stopwords 불러오기

stopwords_df = pd.read_json('stopwords-ko.json')
stopwords=list(stopwords_df[0])
stopwords

In [ ]:
# 영화 별로 리뷰에서 나온 단어들을 빈도수 순서대로 정렬하고 dictionary 형식으로 저장하여 'word_dic' column에 추가

def get_wordcounts(movie_name):
    twitter = Twitter()
    idx = df.index[df['영화명'] == movie_name]
    review_text = list(df['영화리뷰'])[idx[0]]
    raw_pos_tagged = twitter.pos(review_text, norm=True, stem=True) # POS Tagging

    # 영화 제목에 나오는 단어, 주제와 관련없는 의미없는 단어 등을 del_list에 저장
    movie_name_keywords = list(dict(twitter.pos(movie_name, norm=True, stem=True)).keys())
    common_del_list = ['되어다','때문','그리고','좋다','자다','없다','같다','싶다','보다', '하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다'] 
    movie_del_list = ['영화', '배우','연기','개봉','출연','스틸컷','나오다','리뷰']
    del_list = stopwords + movie_name_keywords + eval(df['배우목록'][idx[0]]) + common_del_list + movie_del_list

    # 의미없는 단어들을 모두 제외한 word_cleaned list 만들기
    word_cleaned = []
    for word in raw_pos_tagged: #  ('서울', 'Noun'),
        if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
            if (len(word[0]) != 1) & (word[0] not in del_list) & (word[0] not in 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ'): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
                word_cleaned.append(word[0])

    # 만들어진 word_cleaned list 속 단어들의 빈도수 계산 후 dict 형식으로 저장
    result = Counter(word_cleaned)
    word_dic = dict(result)

    # 빈도 수에 따라 단어들 내림차순으로 정리
    sorted_word_dic = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
    
    # 'word_dic' column에 dict 형식으로 입력
    df['word_dic'][idx[0]] = dict(sorted_word_dic)

In [ ]:
for movie_name in df['영화명']:
    get_wordcounts(movie_name)
  
df['word_dic']

In [ ]:
# 시간 관계 상 4인이 연도별로 따로 get_wordcounts() 함수를 돌린 후 합침
df_0408 = pd.read_csv('df_0408.csv')
df_0913 = pd.read_csv('df_0913.csv')
df_1417 = pd.read_csv('df_1417.csv')
df_1820 = pd.read_csv('df_1820.csv')

word_dic_0408 = list(df_0408[(df_0408['개봉년']>2002) & (df_0408['개봉년']<2009)]['word_dic'])
word_dic_0913 = list(df_0408[(df_0913['개봉년']>2008) & (df_0913['개봉년']<2014)]['word_dic'])
word_dic_1417 = list(df_1417[(df_1417['개봉년']>2013) & (df_1417['개봉년']<2018)]['word_dic'])
word_dic_1820 = list(df_1820[(df_1820['개봉년']>2017) & (df_1820['개봉년']<2021)]['word_dic'])

word_dic = word_dic_0408 + word_dic_0913 + word_dic_1417 + word_dic_1820

df['word_dic'] = word_dic

In [ ]:
# '영화명','개봉년','영화코드','영화리뷰url','영화리뷰','배우목록', 'word_dic' 을 column으로 하는 dataframe 'count_df.csv'로 저장
df.to_csv('count_df.csv')

In [ ]:
df = pd.read_csv('count_df.csv')

In [ ]:
# 미쳐 지우지 못한 의미없는 단어들 delete_list에 추가하여 추후 삭제.
delete_list = {'늘다','보기','많이','많다','만들다','우리','크다','인물','작품','자신','하지만','느낌','가장','대해','보고','모습','사실','알다','그렇게','주인공','이야기','스토리','버리다','보여주다','정도','보이다','많이','극장','흥행','가다','들다','관객','모르다','느끼다','오다','위해','주다','받다','내다','그것','이렇게'}

# year을 입력하면 당 해의 영화들의 word_dic을 병합하여(중복되는 단어의 빈도수를 합함) dict형식으로 return
# 영화명을 입력하면 해당되는 word_dic을 return  
def get_word_dic(para):
    if type(para) == int :
        word_dic = {}
        for dic in df['word_dic'][df['개봉년']==para]:
            try:
                for word in list(eval(dic).keys())[:50]:
                    if word not in delete_list:
                        if word not in list(word_dic.keys()):
                            word_dic[word] = eval(dic)[word]  
                        else:
                            word_dic[word] += eval(dic)[word]
            except:
                continue
        print('{}년의 sorted word dictionary가 생성되었습니다'.format(str(para)))
    
    elif type(para) == str :
        word_dic = df['word_dic'][df['영화명']==para]
        print('영화 {}의 sorted word dictionary가 생성되었습니다'.format(para))
    
    return word_dic

In [ ]:
# 년도 혹은 영화명과 원하는 색깔(colormap형식으로)을 입력하면, 해당되는 리뷰들의 wordcloud 생성, image 파일로 저장

def wordcloud(para, color) :
    peninsula_coloring = np.array(Image.open("peninsula.jpg"))
    
    word_cloud = WordCloud(font_path="BMEuljiro10yearslater.ttf", # font_path="C:/Windows/Fonts/NanumSquareB.ttf"
                           width=2000, height=1000, # 이 부분을 수정하시면 실제 워드클라우드의 크기가 바뀝니다 (해상도가 바뀝니다)
                           # prefer_horizontal= 1.0, # 이 부분의 주석을 해제하시면 단어들이 가로로만 그려지게 됩니다. (0~1)
                           max_words=70, # 단어 70개만 표시
                           mask=peninsula_coloring,
                           background_color='black').generate_from_frequencies(get_word_dic(para)) # 단어 빈도수 기준으로 워드클라우드 제작

    plt.figure(figsize=(15,15)) # 이 부분을 수정하시면 화면에서 보여지는 워드클라우드의 크기가 바뀝니다 
    plt.imshow(word_cloud.recolor(colormap=color)) # 워드 색깔 ('Reds', 'Blues')
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()
    plt.savefig('wordcloud_{}.png'.format(para))

In [ ]:
left = [2004,2005,2006,2007,2017,2018,2019,2020] # 진보(노무현, 문재인 정권)
right = [2008,2009,2010,2011,2012,2013,2014,2015,2016] # 보수(이명박, 박근혜 정권)

for year in left:
    wordcloud(year,'Blues')
for year in right:
    wordcloud(year,'Reds')